# Civil cases

In [1]:
import os
import datetime

import numpy as np
import pandas as pd

from dateutil.relativedelta import relativedelta

In [2]:
pd.set_option('display.width', 250)
pd.set_option('display.max_colwidth', 120)

### Reading & parsing CSV

In [3]:
data_dir = "data"
file_name = "Izskatitas_C1_Kat3_2017_lietas_07.csv"
data_source_path = os.path.join(data_dir, file_name)

In [4]:
df = pd.read_csv(data_source_path, 
                 parse_dates=["Ienākšanas datums", "Izskatīšanas datums"],
                 dtype={"Dzimšanas datums": str},
                 index_col="Lietas ID (caseID)")

In [5]:
def make_float(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

def fix_century(x):
    if x > datetime.datetime(2000, 1, 1):
        return x - relativedelta(years=100)
    else:
        return x

In [6]:
df["Prasījuma apmērs"] = df.apply(lambda item: make_float(item["Prasījuma apmērs"]), axis=1)

df["Būtība"] = df["Būtība"].astype(str)
df["Tiesas ID (courtID)"] = df["Tiesas ID (courtID)"].astype(str)

df["Dzimšanas datums"] = pd.to_datetime(df["Dzimšanas datums"], format="%d%m%y", errors="coerce")
df["Dzimšanas datums"] = df.apply(lambda item: fix_century(item["Dzimšanas datums"]), axis=1)

### Overall stats

In [7]:
df.shape

(16455, 17)

In [8]:
df.head()

,Tiesas ID (courtID),Tiesa,Dzimšanas datums,Ienākšanas kārtība,Ienākšanas datums,Izskatīšanas datums,Būtība,Prasījuma apmērs,atb_fiz,atb_jur,pras_fiz,pras_jur,Nozīmētās sēdes,Notikušās sēdes,naudaS_sods,dzivo_arzemes,atbr_no_nodevas
Lietas ID (caseID),,,,,,,,,,,,,,,,,
213477,40,Valkas rajona tiesa,1959-01-06,Pirmo reizi,2003-06-05,2017-01-23,parāda piedziņa,0.00,0,1,0,1,3,0,0,1,0
448971,32,Rīgas pilsētas Vidzemes priekšpilsētas tiesa,1967-07-05,Pirmo reizi,2005-11-29,2017-04-06,zaudējumu piedziņa,0.00,1,0,1,0,9,1,0,0,1
477470,279,Rīgas pilsētas Pārdaugavas tiesa,1951-05-01,Sakarā ar tiesu apvienošanu,2017-01-31,2017-10-02,parāda piedziņa,21414.34,2,0,1,0,5,3,0,0,0
575613,27,Liepājas tiesa,1963-04-03,Pirmo reizi,2007-06-04,2017-10-26,zaudējumu piedziņa,916.25,6,0,2,0,4,0,0,0,0
580131,32,Rīgas pilsētas Vidzemes priekšpilsētas tiesa,1974-01-25,Pēc piekritības no citas tiesas,2015-10-07,2017-11-13,parāda piedziņa,1588192.95,4,0,1,0,8,1,0,0,0


In [9]:
df.dtypes

Tiesas ID (courtID)            object
Tiesa                          object
Dzimšanas datums       datetime64[ns]
Ienākšanas kārtība             object
Ienākšanas datums      datetime64[ns]
Izskatīšanas datums    datetime64[ns]
Būtība                         object
Prasījuma apmērs              float64
atb_fiz                         int64
atb_jur                         int64
pras_fiz                        int64
pras_jur                        int64
Nozīmētās sēdes                 int64
Notikušās sēdes                 int64
naudaS_sods                     int64
dzivo_arzemes                   int64
atbr_no_nodevas                 int64
dtype: object

In [10]:
df.isnull().sum()

Tiesas ID (courtID)    0
Tiesa                  0
Dzimšanas datums       0
Ienākšanas kārtība     0
Ienākšanas datums      0
Izskatīšanas datums    0
Būtība                 0
Prasījuma apmērs       9
atb_fiz                0
atb_jur                0
pras_fiz               0
pras_jur               0
Nozīmētās sēdes        0
Notikušās sēdes        0
naudaS_sods            0
dzivo_arzemes          0
atbr_no_nodevas        0
dtype: int64

In [11]:
def categorical_stats(df):
    values_df = df.select_dtypes(include="object").nunique().to_frame(name="unique")
    values_df["values"] = values_df.apply(lambda item: df[item.name].unique().tolist(), axis=1)
    return values_df

In [12]:
categorical_stats(df)

,unique,values
Tiesas ID (courtID),30,"[40, 32, 279, 27, 25, 30, 26, 4, 5, 29, 34, 273, 19, 39, 23, 6, 16, 28, 1, 38, 10, 18, 2, 33, 8, 22, 7, 31, 37, 284]"
Tiesa,30,"[Valkas rajona tiesa, Rīgas pilsētas Vidzemes priekšpilsētas tiesa, Rīgas pilsētas Pārdaugavas tiesa, Liepājas tiesa..."
Ienākšanas kārtība,6,"[Pirmo reizi, Sakarā ar tiesu apvienošanu, Pēc piekritības no citas tiesas, Pēc lēmuma atcelšanas, CPL 32.1 panta kā..."
Būtība,1254,"[parāda piedziņa, zaudējumu piedziņa, par rokas naudas piedziņu, par parāda piedziņu, parāda piedziņa, prasības nodr..."


### Cleanup

In [13]:
df.dropna(subset=["Prasījuma apmērs"], inplace=True)
df.drop(labels=["Tiesa"], axis=1, inplace=True)

### Transformations

In [14]:
df["Būtība"] = df.apply(lambda item: " ".join(item["Būtība"].split()), axis=1)

In [15]:
categorical_stats(df)

,unique,values
Tiesas ID (courtID),30,"[40, 32, 279, 27, 25, 30, 26, 4, 5, 29, 34, 273, 19, 39, 23, 6, 16, 28, 1, 38, 10, 18, 2, 33, 8, 22, 7, 31, 37, 284]"
Ienākšanas kārtība,6,"[Pirmo reizi, Sakarā ar tiesu apvienošanu, Pēc piekritības no citas tiesas, Pēc lēmuma atcelšanas, CPL 32.1 panta kā..."
Būtība,1158,"[parāda piedziņa, zaudējumu piedziņa, par rokas naudas piedziņu, par parāda piedziņu, parāda piedziņa, prasības nodr..."


In [16]:
df.rename({"atbr_no_nodevas": "atbr_no_nodevas_known"}, axis=1, inplace=True)

In [17]:
df["age"] = df.apply(lambda item: relativedelta(item["Ienākšanas datums"], item["Dzimšanas datums"]).years, axis=1)
df.drop(labels=["Dzimšanas datums"], axis=1, inplace=True)

In [18]:
df["creation_y"] = df.apply(lambda item: item["Ienākšanas datums"].year, axis=1)
df["creation_m"] = df.apply(lambda item: item["Ienākšanas datums"].month, axis=1)
df["creation_d"] = df.apply(lambda item: item["Ienākšanas datums"].day, axis=1)

df["review_y"] = df.apply(lambda item: item["Izskatīšanas datums"].year, axis=1)
df["review_m"] = df.apply(lambda item: item["Izskatīšanas datums"].month, axis=1)
df["review_d"] = df.apply(lambda item: item["Izskatīšanas datums"].day, axis=1)

In [19]:
def difference_in_days(d1, d2):
    rd = relativedelta(d1, d2)
    now = datetime.datetime.now()
    then = now - rd
    diff = now - then
    return diff.days

In [20]:
df["case_lifetime_days"] = df.apply(
    lambda item: difference_in_days(item["Izskatīšanas datums"], item["Ienākšanas datums"]), axis=1)

In [21]:
df.drop(labels=["Ienākšanas datums", "Izskatīšanas datums"], axis=1, inplace=True)

In [22]:
df.head()

,Tiesas ID (courtID),Ienākšanas kārtība,Būtība,Prasījuma apmērs,atb_fiz,atb_jur,pras_fiz,pras_jur,Nozīmētās sēdes,Notikušās sēdes,...,dzivo_arzemes,atbr_no_nodevas_known,age,creation_y,creation_m,creation_d,review_y,review_m,review_d,case_lifetime_days
Lietas ID (caseID),,,,,,,,,,,,,,,,,,,,,
213477,40,Pirmo reizi,parāda piedziņa,0.00,0,1,0,1,3,0,...,1,0,44,2003,6,5,2017,1,23,4978
448971,32,Pirmo reizi,zaudējumu piedziņa,0.00,1,0,1,0,9,1,...,0,1,38,2005,11,29,2017,4,6,4146
477470,279,Sakarā ar tiesu apvienošanu,parāda piedziņa,21414.34,2,0,1,0,5,3,...,0,0,65,2017,1,31,2017,10,2,245
575613,27,Pirmo reizi,zaudējumu piedziņa,916.25,6,0,2,0,4,0,...,0,0,44,2007,6,4,2017,10,26,3795
580131,32,Pēc piekritības no citas tiesas,parāda piedziņa,1588192.95,4,0,1,0,8,1,...,0,0,41,2015,10,7,2017,11,13,767
